In [1]:
from qiskit import (QuantumCircuit,execute,Aer)
from qiskit.visualization import plot_histogram
from qiskit import*
from django.db import models
from docplex.mp.model import Model                                                                                                      
import docplex
import numpy as np
from itertools import permutations
import gzip
import pandas as pd
# import qiskit.optimization.ising.tsp.TspData as tp
from qiskit.optimization.ising.tsp import TspData
# from qiskit.aqua.optimization import TspData
import matplotlib.pyplot as plt
import matplotlib.axes as axes
%matplotlib inline
import numpy as np
import networkx as nx
import time
from qiskit import BasicAer
from qiskit.tools.visualization import plot_histogram
from qiskit.optimization.ising import tsp
from qiskit.aqua.algorithms import VQE, ExactEigensolver
from qiskit.aqua.algorithms import QAOA, ExactEigensolver

from qiskit.aqua.components.optimizers import SPSA
from qiskit.aqua.components.variational_forms import RY
from qiskit.aqua import QuantumInstance
from qiskit.optimization.ising.common import sample_most_likely
from qiskit.aqua.components.optimizers import COBYLA

# setup aqua logging
import logging
from qiskit.aqua import set_qiskit_aqua_logging
from qiskit import IBMQ

from qiskit import ClassicalRegister, QuantumRegister, QuantumCircuit
from qiskit import execute
from qiskit.tools.jupyter import *
from qiskit.providers.ibmq import least_busy

In [4]:
from qiskit import IBMQ
IBMQ.save_account('faf8747c025d9895e5d8273432c97ba5fb82e352200b4a153707adff0a5620e06b1f760fbf9a3a9988486f23698bc537c5e9d82979ee4ba699618a43eee1130d',overwrite = True)

In [5]:
IBMQ.load_account()

<AccountProvider for IBMQ(hub='ibm-q', group='open', project='main')>

In [6]:
G = []

In [7]:
p = [4,5,6,7,8,9,10,11]
q = [i**(2) for i in p ]
m = 0
v = open("Matrices.txt"  , "r")
w = v.read().split()
for i in range (len(w)):
    w[i] = int(float(w[i]))
for i in range (len(q)):
    G.append(np.reshape(w[m:m+q[i]] , (p[i] , p[i])))
    m = m + q[i]
print('Completed Successfully')

Completed Successfully


In [11]:
#Using Aer Backends
def QAOA1(m,qubitOp , a , v):
    seed = 1059
    start = time.time()
    spsa = SPSA(max_trials=m)
    vqe =  QAOA(qubitOp,spsa ,p = a)

    backend = BasicAer.get_backend('statevector_simulator')

    quantum_instance = QuantumInstance(backend, seed_simulator=seed, seed_transpiler=seed,shots = v)

    result = vqe.run(quantum_instance)
    x = sample_most_likely(result['eigvecs'][0])
    e = tsp.tsp_feasible(x)
    return e,result

In [12]:
x = TspData('tmp',len(G[0]),np.zeros((3,3)),G[0])
qubitOp ,offset = tsp.get_operator(x)
QAOA1(300,qubitOp , 1 , 1024)

In [ ]:
#Grid Search QAOA
O =np.zeros((2,4))
l = np.arange(100,300,20) #max trails
s = np.arange(100, 1024 , 300) # shots
q = np.arange(1,4)#p value
for k in range(1):
    f = []
    for i in range(len(l)):
        for j in range(len(s)):
            for t in range(len(q)):
                h = G[0]
                x = TspData('tmp',len(h),np.zeros((3,3)),h)
                qubitOp ,offset = tsp.get_operator(x)
                r = time.clock()
                a = QAOA1(int(l[i]), qubitOp, int(q[t]), s[j])
                w = time.clock()
                f.append((a, l[i] , q[t],s[j]))
                print('Iteration:',i,j,t,w-r)
    p = 0
    for m in f:
        if m[0] == True and p != -1:
            print('Feasible')
            O[k,0] = len(V[k])
            #max trails
            O[k,1] = m[1]
            #shots
            O[k,2] = m[2]
            #p value
            O[k,3] = m[3]
            p = -1
        else:
            print('Infeasible')


In [ ]:
#Using quantum devices
def QAOA2(m,qubitOp , a , v):
    seed = 1059
    start = time.time()
    spsa = SPSA(max_trials=m)
    vqe =  QAOA(qubitOp,spsa ,p = a)

    my_provider = IBMQ.get_provider('ibm-q-wits')
    my_provider.backends()
    device = my_provider.get_backend('ibmq_cambridge')

    quantum_instance = QuantumInstance(device, seed_simulator=seed, seed_transpiler=seed , skip_qobj_validation = False,shots = v )

    result = vqe.run(quantum_instance)
    x = sample_most_likely(result['eigvecs'][0])
    e = tsp.tsp_feasible(x)
    return e, result

In [ ]:
x = TspData('tmp',len(G[0]),np.zeros((3,3)),G[0])
qubitOp ,offset = tsp.get_operator(x)
QAOA2(300,qubitOp , 1 , 1024)